In [1]:
!pip3 install setproctitle


  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp36-cp36m-linux_x86_64.whl size=33925 sha256=93ef33ce4296345c3ced8ff94cfb9abe6bd8bd7afe9efb2016e6d042709d2f58
  Stored in directory: /root/.cache/pip/wheels/e6/b1/a6/9719530228e258eba904501fef99d5d85c80d52bd8f14438a3
Successfully built setproctitle


In [2]:
import torch
print(torch.__version__)

1.5.0+cu101


In [0]:
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
# from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

import sys
import math

class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4*growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2

        nChannels = 2*growthRate
        self.conv1 = nn.Conv2d(3, nChannels, kernel_size=3, padding=1,
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans1 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        nChannels += nDenseBlocks*growthRate

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        out = F.log_softmax(self.fc(out))
        return out


In [0]:
# From https://gist.github.com/apaszke/01aae7a0494c55af6242f06fad1f8b70
from graphviz import Digraph
from torch.autograd import Variable

def save(fname, creator):
    dot = Digraph(comment='LRP',
                node_attr={'style': 'filled', 'shape': 'box'})
    #, 'fillcolor': 'lightblue'})

    seen = set()

    def add_nodes(var):
        if var not in seen:
            if isinstance(var, Variable):
                dot.node(str(id(var)), str(var.size()), fillcolor='lightblue')
            else:
                dot.node(str(id(var)), type(var).__name__)
            seen.add(var)
            if hasattr(var, 'previous_functions'):
                for u in var.previous_functions:
                    dot.edge(str(id(u[0])), str(id(var)))
                    add_nodes(u[0])

    add_nodes(creator)
    dot.save(fname)


In [5]:
#!/usr/bin/env python3

# import argparse
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
# from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader

import os
import sys
import math

import shutil

import setproctitle

# import densenet
# import make_graph

def main():
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--batchSz', type=int, default=64)
    # parser.add_argument('--nEpochs', type=int, default=300)
    # parser.add_argument('--no-cuda', action='store_true')
    # parser.add_argument('--save')
    # parser.add_argument('--seed', type=int, default=1)
    # parser.add_argument('--opt', type=str, default='sgd',
    #                     choices=('sgd', 'adam', 'rmsprop'))
    # args = parser.parse_args()

    # args.cuda = not args.no_cuda and torch.cuda.is_available()
    # args.save = args.save or 'work/densenet.base'
    # setproctitle.setproctitle(args.save)
    
    # torch.manual_seed(args.seed)
    # if args.cuda:
    #     torch.cuda.manual_seed(args.seed)

    # if os.path.exists(args.save):
    #     shutil.rmtree(args.save)
    # os.makedirs(args.save, exist_ok=True)

    normMean = [0.49139968, 0.48215827, 0.44653124]
    normStd = [0.24703233, 0.24348505, 0.26158768]
    normTransform = transforms.Normalize(normMean, normStd)

    trainTransform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normTransform
    ])
    testTransform = transforms.Compose([
        transforms.ToTensor(),
        normTransform
    ])
    device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
    # kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
    trainLoader = DataLoader(
        dset.CIFAR10(root='cifar', train=True, download=True,
                     transform=trainTransform),
        batch_size=64, shuffle=True, num_workers=4)
    testLoader = DataLoader(
        dset.CIFAR10(root='cifar', train=False, download=True,
                     transform=testTransform),
        batch_size=64, shuffle=False, num_workers=4)

    net = DenseNet(growthRate=12, depth=100, reduction=0.5,
                            bottleneck=True, nClasses=10)

    print('  + Number of params: {}'.format(
        sum([p.data.nelement() for p in net.parameters()])))
    # if args.cuda:
    net = net.to(device)

    # if args.opt == 'sgd':
    # optimizer = optim.SGD(net.parameters(), lr=1e-1,
                            # momentum=0.9, weight_decay=1e-4)
    # elif args.opt == 'adam':
    optimizer = optim.Adam(net.parameters(), weight_decay=1e-4)
    # elif args.opt == 'rmsprop':
    #     optimizer = optim.RMSprop(net.parameters(), weight_decay=1e-4)

    # trainF = open(os.path.join(args.save, 'train.csv'), 'w')
    # testF = open(os.path.join(args.save, 'test.csv'), 'w')

    for epoch in range(1, 300 + 1):
        # adjust_opt('sgd', optimizer, epoch)
        train(epoch, net, trainLoader, optimizer,device)
        test(epoch, net, testLoader, optimizer,device)
        # torch.save(net, os.path.join(args.save, 'latest.pth'))
        # os.system('./plot.py {} &'.format(args.save))

    # trainF.close()
    # testF.close()

def train(epoch, net, trainLoader, optimizer,device):
    net.train()
    nProcessed = 0
    nTrain = len(trainLoader.dataset)
    for batch_idx, (data, target) in enumerate(trainLoader):
        # if args.cuda:
        data, target = data.to(device), target.to(device)
        # data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        # make_graph.save('/tmp/t.dot', loss.creator); assert(False)
        loss.backward()
        optimizer.step()
        nProcessed += len(data)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect = pred.ne(target.data).cpu().sum()
        err = 100.*incorrect/len(data)
        partialEpoch = epoch + batch_idx / len(trainLoader) - 1
        print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
            partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(trainLoader),
            loss.item(), err))

        # trainF.write('{},{},{}\n'.format(partialEpoch, loss.data[0], err))
        # trainF.flush()

def test(epoch, net, testLoader, optimizer,device):
    net.eval()
    test_loss = 0
    incorrect = 0
    for data, target in testLoader:
        # if args.cuda:
        data, target = data.to(device), target.to(device)
        # data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.nll_loss(output, target).item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        incorrect += pred.ne(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader.dataset)
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))

    # testF.write('{},{},{}\n'.format(epoch, test_loss, err))
    # testF.flush()

def adjust_opt(optAlg, optimizer, epoch):
    if optAlg == 'sgd':
        if epoch < 150: lr = 1e-1
        elif epoch == 150: lr = 1e-2
        elif epoch == 225: lr = 1e-3
        else: return

        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

# if __name__=='__main__':
# device = torch.device
main()



Test set: Average loss: 0.3667, Error: 1189/10000 (12%)

Train Epoch: 23.00 [64/50000 (0%)]	Loss: 0.128795	Error: 6.250000
Train Epoch: 23.00 [128/50000 (0%)]	Loss: 0.309722	Error: 12.500000
Train Epoch: 23.00 [192/50000 (0%)]	Loss: 0.197141	Error: 7.812500
Train Epoch: 23.00 [256/50000 (0%)]	Loss: 0.177331	Error: 9.375000
Train Epoch: 23.01 [320/50000 (1%)]	Loss: 0.192000	Error: 6.250000
Train Epoch: 23.01 [384/50000 (1%)]	Loss: 0.269335	Error: 9.375000
Train Epoch: 23.01 [448/50000 (1%)]	Loss: 0.341553	Error: 14.062500
Train Epoch: 23.01 [512/50000 (1%)]	Loss: 0.212444	Error: 10.937500
Train Epoch: 23.01 [576/50000 (1%)]	Loss: 0.249649	Error: 4.687500
Train Epoch: 23.01 [640/50000 (1%)]	Loss: 0.238301	Error: 7.812500
Train Epoch: 23.01 [704/50000 (1%)]	Loss: 0.329480	Error: 10.937500
Train Epoch: 23.01 [768/50000 (1%)]	Loss: 0.191726	Error: 6.250000
Train Epoch: 23.02 [832/50000 (2%)]	Loss: 0.114555	Error: 3.125000
Train Epoch: 23.02 [896/50000 (2%)]	Loss: 0.149550	Error: 6.250000
T

KeyboardInterrupt: ignored

In [0]:
!nvidia-smi

Sun Jun 14 15:11:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    72W / 149W |  11437MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      9 ?        00:00:02 node
     19 ?        00:00:04 jupyter-noteboo
    110 ?        00:00:00 tail
    119 ?        00:04:35 /usr/bin/python
    506 ?        00:00:00 ps


In [0]:
! kill 19